In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls

gdrive	sample_data


In [0]:
DIR = '/content/gdrive/My Drive/Spark/data/'

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName('Python Spark regression example')\
        .config('spark.some.config.option','some-value')\
        .getOrCreate()

In [12]:
file_loc = DIR + 'Advertising.csv'
#print(file_loc)
df = spark.read.csv(file_loc, inferSchema=True, header=True)
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [13]:
df.show()
df.printSchema()

+---+-----+-----+---------+-----+
|_c0|   TV|Radio|Newspaper|Sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
|  4|151.5| 41.3|     58.5| 18.5|
|  5|180.8| 10.8|     58.4| 12.9|
|  6|  8.7| 48.9|     75.0|  7.2|
|  7| 57.5| 32.8|     23.5| 11.8|
|  8|120.2| 19.6|     11.6| 13.2|
|  9|  8.6|  2.1|      1.0|  4.8|
| 10|199.8|  2.6|     21.2| 10.6|
| 11| 66.1|  5.8|     24.2|  8.6|
| 12|214.7| 24.0|      4.0| 17.4|
| 13| 23.8| 35.1|     65.9|  9.2|
| 14| 97.5|  7.6|      7.2|  9.7|
| 15|204.1| 32.9|     46.0| 19.0|
| 16|195.4| 47.7|     52.9| 22.4|
| 17| 67.8| 36.6|    114.0| 12.5|
| 18|281.4| 39.6|     55.8| 24.4|
| 19| 69.2| 20.5|     18.3| 11.3|
| 20|147.3| 23.9|     19.1| 14.6|
+---+-----+-----+---------+-----+
only showing top 20 rows

root
 |-- _c0: integer (nullable = true)
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- 

In [0]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
# I provide two ways to build the features and labels
# method 1 (good for small feature):
#def transData(row):
# return Row(label=row["Sales"],
# features=Vectors.dense([row["TV"],
# row["Radio"],
# row["Newspaper"]]))
# Method 2 (good for large features):

def transData(data):
  return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

In [27]:
transformed= transData(df)
transformed.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,230.1,37.8,6...| 22.1|
|[2.0,44.5,39.3,45.1]| 10.4|
|[3.0,17.2,45.9,69.3]|  9.3|
|[4.0,151.5,41.3,5...| 18.5|
|[5.0,180.8,10.8,5...| 12.9|
+--------------------+-----+
only showing top 5 rows



In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer(inputCol = 'features', \
                               outputCol = 'indexedFeatures', \
                               maxCategories = 4).fit(transformed)

data = featureIndexer.transform(transformed)              

In [29]:
data.show(5,True)

+--------------------+-----+--------------------+
|            features|label|     indexedFeatures|
+--------------------+-----+--------------------+
|[1.0,230.1,37.8,6...| 22.1|[1.0,230.1,37.8,6...|
|[2.0,44.5,39.3,45.1]| 10.4|[2.0,44.5,39.3,45.1]|
|[3.0,17.2,45.9,69.3]|  9.3|[3.0,17.2,45.9,69.3]|
|[4.0,151.5,41.3,5...| 18.5|[4.0,151.5,41.3,5...|
|[5.0,180.8,10.8,5...| 12.9|[5.0,180.8,10.8,5...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [0]:
# Split the data

(trainingData, testData) = transformed.randomSplit([0.6,0.4])

In [31]:
trainingData.show(5)
testData.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,230.1,37.8,6...| 22.1|
|[2.0,44.5,39.3,45.1]| 10.4|
|[3.0,17.2,45.9,69.3]|  9.3|
|[4.0,151.5,41.3,5...| 18.5|
|[5.0,180.8,10.8,5...| 12.9|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
| [6.0,8.7,48.9,75.0]|  7.2|
|[8.0,120.2,19.6,1...| 13.2|
| [14.0,97.5,7.6,7.2]|  9.7|
|[15.0,204.1,32.9,...| 19.0|
|[18.0,281.4,39.6,...| 24.4|
+--------------------+-----+
only showing top 5 rows



In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression()
pipeline = Pipeline(stages=[featureIndexer,lr])

model = pipeline.fit(trainingData)

In [34]:
predictions = model.transform(testData)
predictions.select('features','label','prediction').show(5)

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
| [6.0,8.7,48.9,75.0]|  7.2|12.701725365769034|
|[8.0,120.2,19.6,1...| 13.2|12.013634444861534|
| [14.0,97.5,7.6,7.2]|  9.7| 8.680882159114507|
|[15.0,204.1,32.9,...| 19.0|18.359011345429966|
|[18.0,281.4,39.6,...| 24.4| 23.12274437377659|
+--------------------+-----+------------------+
only showing top 5 rows



In [35]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol='label',predictionCol='prediction',
                                metricName='rmse')

rmse = evaluator.evaluate(predictions)
print('Root Mean Squared Error (RMSE) on test data = %g' %rmse)

Root Mean Squared Error (RMSE) on test data = 1.75977


In [0]:
y_true = predictions.select('label').toPandas()
y_pred = predictions.select('prediction').toPandas()

import sklearn.metrics

r2_score = sklearn.metrics.r2_score(y_true,y_pred)
print('r2_score: {0}'.format(r2_score))